<a href="https://colab.research.google.com/github/deeplearningexplore/siamese/blob/main/question_pairs_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# !pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c quora-question-pairs

Saving kaggle.json to kaggle.json
  0% 0.00/4.95M [00:00<?, ?B/s]
100% 4.95M/4.95M [00:00<00:00, 81.3MB/s]
 99% 113M/114M [00:01<00:00, 94.1MB/s] 
100% 114M/114M [00:01<00:00, 107MB/s] 
test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
 43% 9.00M/21.2M [00:00<00:00, 71.2MB/s]
100% 21.2M/21.2M [00:00<00:00, 102MB/s] 


In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import zipfile
import pandas as pd

%matplotlib inline

In [4]:
zip = zipfile.ZipFile("test.csv.zip", "r")
zip.extractall("./data")


In [5]:
zip = zipfile.ZipFile("train.csv.zip", "r")
zip.extractall("./data")

zip.close()

In [6]:
!rm "train.csv.zip"
!rm "test.csv.zip"

In [7]:
train_data = pd.read_csv("./data/train.csv")
test_data = pd.read_csv("./data/test.csv")
print(train_data.head())
print(test_data.head())

   id  qid1  ...                                          question2 is_duplicate
0   0     1  ...  What is the step by step guide to invest in sh...            0
1   1     3  ...  What would happen if the Indian government sto...            0
2   2     5  ...  How can Internet speed be increased by hacking...            0
3   3     7  ...  Find the remainder when [math]23^{24}[/math] i...            0
4   4     9  ...            Which fish would survive in salt water?            0

[5 rows x 6 columns]
   test_id  ...                                          question2
0        0  ...  Why did Microsoft choose core m3 and not core ...
1        1  ...        How much cost does hair transplant require?
2        2  ...                      What you send money to China?
3        3  ...                                  What foods fibre?
4        4  ...                     How their can I start reading?

[5 rows x 3 columns]


In [8]:
train_data.columns

Index(['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate'], dtype='object')

In [9]:
train_data = train_data[train_data["is_duplicate"]==1]
train_data.head()

,id,qid1,qid2,question1,question2,is_duplicate
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
11,11,23,24,How do I read and find my YouTube comments?,How can I see all my Youtube comments?,1
12,12,25,26,What can make Physics easy to learn?,How can you make physics easy to learn?,1
13,13,27,28,What was your first sexual experience like?,What was your first sexual experience?,1


In [10]:
train_d1 = train_data['question1'].values
train_d2 = train_data['question2'].values

In [11]:
train_d1.shape

(149263,)

In [12]:
train_d = np.concatenate([train_d1,train_d2])

In [13]:
train_d.shape

(298526,)

In [14]:
oov_token = "<ukw>"
pad = "post"


In [15]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token=oov_token)
tokenizer.fit_on_texts(train_d)
word_index = tokenizer.word_index


In [16]:
len(word_index)

31468

In [17]:
train_data1 = tokenizer.texts_to_sequences(train_d1)

In [18]:
train_data2 = tokenizer.texts_to_sequences(train_d2)

In [19]:
word_index["ambulance"]

19614

In [20]:
def data_generator(q1,q2,batch_size):
  indexes = list(range(len(q1)))
  indexes = tf.random.shuffle(indexes)
  length = len(q1)
  input1 = []
  input2 = []
  index = 0

  while True:
    if index==length:
      index = 0
      indexes = tf.random.shuffle(indexes)
    
    input1.append(q1[indexes[index]])
    input2.append(q2[indexes[index]])

    index+=1

    if len(input1)==batch_size:
      max_length = np.max((np.max([len(x) for x in input1]),np.max([len(x) for x in input2])))
      max_length = 2**int(np.ceil(np.log2(max_length)))

      input1 = tf.keras.preprocessing.sequence.pad_sequences(input1,maxlen=max_length,padding=pad,truncating=pad)
      input2 = tf.keras.preprocessing.sequence.pad_sequences(input2,maxlen=max_length,padding=pad,truncating=pad)

      yield np.array(input1),np.array(input2)

      input1 = []
      input2 = []



In [21]:
a,b=next(data_generator(train_data1,train_data2,5))

In [22]:
print(a[0])
print(b[0])

[   3   25   10  192   95 2448 1263    9   10  455    0    0    0    0
    0    0]
[   3    8  238   95 2448 1263    9   10  429    0    0    0    0    0
    0    0]


In [23]:
!wget "https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip"

--2020-10-13 10:49:00--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 172.67.9.4, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 681808098 (650M) [application/zip]
Saving to: ‘wiki-news-300d-1M.vec.zip’

wiki-news-300d-1M.v 100%[===================>] 650.22M  24.7MB/s    in 29s     

2020-10-13 10:49:29 (22.5 MB/s) - ‘wiki-news-300d-1M.vec.zip’ saved [681808098/681808098]



In [24]:
zip = zipfile.ZipFile("wiki-news-300d-1M.vec.zip", "r")
zip.extractall("./data")
zip.close()

In [25]:
!rm "wiki-news-300d-1M.vec.zip"

In [26]:
import io

def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = np.asarray(tokens[1:], dtype='float32')
    return data

embedding_vectors =  load_vectors("data/wiki-news-300d-1M.vec")

In [27]:
len(embedding_vectors)

999994

In [28]:
vocab_size =  len(tokenizer.word_index) + 1
dimensions = 300
batch_size = 128

In [29]:
embedding_matrix = np.zeros((vocab_size, dimensions))
for word,index in word_index.items():
  val = embedding_vectors.get(word, None)
  if val is not None:
    embedding_matrix[index] = val

In [30]:
del embedding_vectors

In [31]:
embedding_layer = tf.keras.layers.Embedding(vocab_size,
                                            dimensions,
                                            embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
                                            trainable = False
                                            )

model = tf.keras.Sequential()
model.add(embedding_layer)
model.add(tf.keras.layers.LSTM(10))

input1 = tf.keras.Input(shape=(None,))
input2 = tf.keras.Input(shape=(None,))

vectors1 = model(input1)
vectors2 = model(input2)

saimese = tf.keras.Model(inputs = [input1, input2], outputs = [vectors1, vectors2])

saimese.summary()


Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 10)           9453140     input_1[0][0]                    
                                                                 input_2[0][0]                    
Total params: 9,453,140
Trainable params: 12,440
Non-trainable params: 9,440,700
__________________________________________________________________________________________________


In [32]:
x = np.array(saimese([a,b]))

In [33]:
def cosine_similarity(a,b):
  return np.dot(a,b)/(np.linalg.norm(a) * np.linalg.norm(b))

In [42]:
def TripletLoss(vectors, margin = 0.25):
  anchor1 = vectors[0,:,:]
  anchor2 = vectors[1,:,:]
  batch_size = anchor1.shape[0]
  similarity_matrix = np.zeros((batch_size, batch_size))
  for i,vec1 in enumerate(anchor1):
    for j,vec2 in enumerate(anchor2):
      similarity_matrix[i,j] = cosine_similarity(vec1, vec2)
  print(similarity_matrix)
  positive = np.diagonal(similarity_matrix)
  temp1 = similarity_matrix - np.eye(batch_size) * 2.0
  temp2 = similarity_matrix * ( 1 - np.eye(batch_size))

  mean_negative = np.sum(temp2) / (batch_size-1)
  max_negative = np.max(temp1, axis=1)

  loss1 = np.maximum(0, mean_negative - positive + margin )
  loss2 = np.maximum(0, max_negative - positive + margin )

  loss = np.mean(loss1 + loss2)

  return loss

u = TripletLoss(x)

[[ 0.9951281   0.86310029  0.42041731  0.05981461 -0.0013075 ]
 [ 0.75661564  0.80265611  0.44160438  0.42660612  0.00786758]
 [ 0.3331874   0.65120471  0.35735729  0.57668418 -0.3675293 ]
 [ 0.15237054  0.22000059 -0.43563113  0.96621895  0.36135826]
 [-0.09621354 -0.49779284 -0.64354414  0.28846842  0.97638059]]


In [44]:
def predict(a, b, margin=0):
  vectors = saimese([a,b])
  similarity = cosine_similarity(vectors[0], vectors[1])
  for i in similarity:
    if i > 0:
      print("similar")
    else: 
      print("Not Similar")


In [47]:
saimese.compile(optimizer="adam", loss = TripletLoss)

In [49]:
saimese.fit_generator(data_generator, epochs=1)

ValueError: ignored